In [ ]:
import cv2
import numpy as np


def Object_detect(new_image, image_template):
    
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    image2 = image_template
    
    orb = cv2.ORB_create(1000, 1.4)

    (kp1, des1) = orb.detectAndCompute(image1, None)
    (kp2, des2) = orb.detectAndCompute(image2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(des1,des2)
    matches = sorted(matches, key=lambda val: val.distance)

    return len(matches)

cap = cv2.VideoCapture(0)

image_template = cv2.imread('images/box_in_scene.png', 0) 
  

while True:
    ret, frame = cap.read()
    
    h, w = frame.shape[:2]

    t_left_x = int(w/3)
    t_left_y = int((h/2) + (h/4))
    b_right_x = int((w/ 3)*2)
    b_right_y = int((h/2) - (h/4))

    cv2.rectangle(frame, (t_left_x,t_left_y), (b_right_x,b_right_y), 255, 2)
    cropped = frame[b_right_y:t_left_y , t_left_x:b_right_x]
    frame = cv2.flip(frame,1)     

    matches = Object_detect(cropped, image_template)
    
    output_string = "Matches = " + str(matches)
    cv2.putText(frame, output_string, (200,460), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    
    threshold = 380    
   
    if matches > threshold:
        cv2.rectangle(frame, (t_left_x,t_left_y), (b_right_x,b_right_y), (0,225,255), 2)
        cv2.putText(frame,'Object Found',(170,45), cv2.FONT_HERSHEY_COMPLEX, 1.4 ,(0,0,255), 2)
    
    cv2.imshow('Object Detector', frame)    
    if cv2.waitKey(1) == 13: 
        break

cap.release()

cv2.destroyAllWindows()   